# Data Analysis:
### This notebook aims to make an initial description of the data available for training in TrainingData_BU&Public_CWS.csv
.

TODO:
- Fix NAs
- Import all the fields as correct types
- Investigate distribution of data. 
- Find out whether the clips are from different sections of a longer recordings.
- Find out whether there are duplicates within the database. This is very important for making sure that the withheld test set is not contaminated by examples which the model has previously seen. 

<br>

### Notes
There is a field 'ARU Task Status' with value of 'bad weather' and 'malfunction'. Option to filter out bad weather days. 






In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_path = Path('../../data/raw/')

In [3]:
meta = pd.read_csv(data_path/'TrainingData_BU&Public_CWS.csv')


/var/folders/hy/36n7jr6x3h74ddx507dc12_00000gn/T/ipykernel_59611/2075253170.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(data_path/'TrainingData_BU&Public_CWS.csv')


# FIx NAs
1. Find out how many values in each field are blank

In [4]:
display_all(meta.tail())

NameError: name 'display_all' is not defined

The last entry in this .csv file contains NaNs for most of the fields. Delete this. 

In [ ]:
meta = meta.drop(1152839)

In [ ]:
features = meta.columns
features[2]

In [ ]:
idx = 28
print(features[idx])
display_all(meta[features[idx]].value_counts(dropna=False))

In [ ]:
meta.dtypes

Some of the data types have multiple types within one field. This is causing duplicate values when viewing data (e.g. abundance is showing a numeric value for 1 and a string value for 1). Different data types are are counted separately when using pandas.describe(). To fix this issue, specify the data type by passing in a type dictionary when reading the csv file. 

In [5]:
def display_all(df):
    with pd.option_context('display.max_columns', 70):
        with pd.option_context("display.max_rows", 1000):
            display(df)


In [6]:
type_dict = {
    "organization": str,
    "project": object,
    "project_id": int,
    "location": str,
    "location_id": int,
    "recording_date_time": object,
    "recording_id": int,
    "task_method": str,
    "task_id": int,
    "aru_task_status": str,
    "species_code": str,
    "species_common_name": str,
    "species_scientific_name": str,
    "species_class": str,
    "detection_time": float,
    "task_duration": float,
    "tag_duration": float,
    "min_tag_freq": object,
    "max_tag_freq": object,
    "tag_id": int,
    "individual_order": str,
    "vocalization": str,
    "abundance": str,
    "tag_rating": float,
    "tag_is_verified": object,
    "clip_channel_used": float,
    "observer": object,
    "observer_user_id": int,
    "verifier_user_id": int,
    "left_full_freq_tag_rms_peak_dbfs": float,
    "left_full_freq_tag_rms_trough_dbfs": float,
    "left_full_freq_tag_pk_count": float,
    "left_full_freq_tag_dc_offset": float,
    "left_full_freq_tag_min_level": float,
    "left_full_freq_tag_max_level": float,
    "left_full_freq_tag_peak_level_dbfs": float,
    "left_freq_filter_tag_rms_peak_dbfs": float,
    "left_freq_filter_tag_rms_trough_dbfs": float,
    "left_freq_filter_tag_pk_count": float,
    "left_freq_filter_tag_dc_offset": float,
    "left_freq_filter_tag_min_level": float,
    "left_freq_filter_tag_max_level": float,
    "left_freq_filter_tag_peak_level_dbfs": float,
    "right_full_freq_tag_rms_peak_dbfs": float,
    "right_full_freq_tag_rms_trough_dbfs": float,
    "right_full_freq_tag_pk_count": float,
    "right_full_freq_tag_dc_offset": float,
    "right_full_freq_tag_min_level": float,
    "right_full_freq_tag_max_level": float,
    "right_full_freq_tag_peak_level_dbfs": float,
    "right_freq_filter_tag_rms_peak_dbfs": float,
    "right_freq_filter_tag_rms_trough_dbfs": float,
    "right_freq_filter_tag_pk_count": float,
    "right_freq_filter_tag_dc_offset": float,
    "right_freq_filter_tag_min_level": float,
    "right_freq_filter_tag_max_level": float,
    "right_freq_filter_tag_peak_level_dbfs": float,
    "tagged_in_wildtrax": str,
    "url": object,
    "tag_spectrogram_url": object,
    "clip_url": object,
    "sensorId": int,
    "tasks": int,
    "status": object,
}

In [7]:
meta.recording_id.value_counts().head()

104940.0    394
104939.0    370
104942.0    314
104941.0    290
72961.0     286
Name: recording_id, dtype: int64

In [ ]:
meta = pd.read_csv(data_path/'TrainingData_BU&Public_CWS.csv', dtype=type_dict)

In [ ]:
meta.head()

The recording ids aren't unique, indicating that there are mulitple references to the same recording within the database.


In [ ]:
projects = meta.project.unique().tolist()

# Filter out old recordings. 
There are 114 projects in the database. Some of the older (pre-2016 - 2017) recording units had a high signal to noise ratio due to the recording devices used. Filtering out projects older than this could improve average quality of recordings. 
<br><br>
Also more recent data might better represent current call patterns in the field. 


In [ ]:
years = range(2020, 2024)

In [ ]:
recent_projects = []
for year in years:
    year_projects = [project for project in projects if f'{year}' in project]
    recent_projects.extend(year_projects)

In [ ]:
recent_projects

In [ ]:
len(meta.recording_id.unique())

There are 97439 unique recordings in the database.


In [ ]:
len(meta[meta.clip_url.isna()])

Some of these clips don't have clip_url links to an audio file. There are a total of 28386 un-linked tags out of a total 1152840 tags.


In [ ]:
def display_all(df):
    with pd.option_context('display.max_columns', 70):
        display(df)

display_all(meta)


In [ ]:
meta

In [ ]:
display_all(meta.loc[meta.species_code=='OSFL'].url.iloc[1])

# Look at the distribution of clip lengths in  TrainingData_BU&Public_CWS.csv

In [ ]:
df = meta.loc[meta.tag_duration<30]

# Create a figure with subplots for both the histogram and KDE plot
fig, ax = plt.subplots()
# Create a histogram
sns.histplot(data=df, x='tag_duration', ax=ax, bins=50)
ax.set_xlabel('Time (s)')
ax.set_ylabel('number of recordings')
ax.set_title('Distribution Plot of sample lengths')

plt.show()

In [ ]:
sns.histplot(data=meta.loc[meta.tag_duration > 40], x='tag_duration', bins=20, log_scale=False)


# Load data from other sources:
### Look at the BU everything forever csv

In [ ]:
!ls $data_path/SpeciesRawDownload

In [ ]:
everythingforever = pd.read_csv(data_path/'SpeciesRawDownload/recording_report.csv')

In [ ]:
display_all(everythingforever)